<a href="https://colab.research.google.com/github/gillbatesiii/m2m-capstone1-borderdata/blob/master/capstone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis on land border crossings from Canada to the US

Hi, I would like to present an analysis on passenger land border crossings from Canada into the US. In particular, I would like to compare the current year's border crossings to previous years.

## Initial setup

Before starting, we will need to install `sodapy`. This is the SDK that will allow us to interact with Socrata data APIs.

Bureau of Transportation Statistics (BTS) data is hosted on the Socrata platform. This allows us to send a query along with the API request, as we will see below.

In [ ]:
!pip install sodapy

In [ ]:
import pandas as pd
from sodapy import Socrata
from typing import Final
import plotly.express as px
from google.colab import userdata


To obtain an app token, you will need to sign up here https://data.bts.gov/signup

This is optional, you will still be able to fetch data without the app token, subject to rate limits.

In [ ]:
# retrieve app token from Google Colab Secrets
try:
  APP_TOKEN = userdata.get('CAPSTONE1_SOCRATA_APP_TOKEN')
except (userdata.SecretNotFoundError, userdata.NotebookAccessError):
  APP_TOKEN = None
  print("Cannot retrieve Socrata app token. This is ok, we can still fetch the data, it will just be subject to rate limits.")


Create the Socrata client that will be used to fetch data from the API.

In [ ]:
client = Socrata("data.bts.gov", APP_TOKEN, timeout=90)


## Data Fetching

We will be using data from the US DOT Bureau of Transportation Statistics. We will be pulling from the Border Crossing Entry Data API
https://data.bts.gov/Research-and-Statistics/Border-Crossing-Entry-Data/keg4-3bc2/about_data


For the purposes of this analysis, we will be retrieving data for the years 2017-2025. Since the dataset contains data dating back to 1994, the payload will be quite large if we fetch it in its entirety. We will use SoQL (Socrata Query Language) to limit the results.


In [ ]:
# Constants
WHERE_CLAUSE: Final = "border = 'US-Canada Border' AND date >= '2017-01-01'"
DATASET_IDENTIFIER: Final = "keg4-3bc2"

# Fetch number of expected results for desired Sodapy query
row_count = client.get(
    DATASET_IDENTIFIER,
    query=f"SELECT count(*) WHERE {WHERE_CLAUSE}",
)[0]["count"]
print("row_count", row_count)

row_count 57534


In [ ]:
row_count = int(row_count)
results = client.get(DATASET_IDENTIFIER, limit=row_count, where=WHERE_CLAUSE)
client.close()



In [ ]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,port_name,state,port_code,border,date,measure,value,latitude,longitude,point
0,International Falls,Minnesota,3604,US-Canada Border,2017-01-01T00:00:00.000,Buses,18,48.6078,-93.401355,"{'type': 'Point', 'coordinates': [-93.401355, ..."
1,Metaline Falls,Washington,3025,US-Canada Border,2017-01-01T00:00:00.000,Personal Vehicles,1857,48.999972,-117.299444,"{'type': 'Point', 'coordinates': [-117.299444,..."
2,Alcan,Alaska,3104,US-Canada Border,2017-01-01T00:00:00.000,Bus Passengers,0,62.614961,-141.001444,"{'type': 'Point', 'coordinates': [-141.001444,..."
3,Westhope,North Dakota,3419,US-Canada Border,2017-01-01T00:00:00.000,Personal Vehicles,490,48.999611,-101.017277,"{'type': 'Point', 'coordinates': [-101.017277,..."
4,Calais,Maine,0115,US-Canada Border,2017-01-01T00:00:00.000,Rail Containers Loaded,60,45.188548,-67.275381,"{'type': 'Point', 'coordinates': [-67.275381, ..."
...,...,...,...,...,...,...,...,...,...,...
57529,Point Roberts,Washington,3017,US-Canada Border,2025-06-01T00:00:00.000,Buses,28,49.0020555547,-123.068055556,"{'type': 'Point', 'coordinates': [-123.0680555..."
57530,Ogdensburg,New York,0701,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,29047,44.7330898624,-75.4577501759,"{'type': 'Point', 'coordinates': [-75.45775017..."
57531,Limestone,Maine,0118,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,1228,46.924555,-67.789597,"{'type': 'Point', 'coordinates': [-67.789597, ..."
57532,Van Buren,Maine,0108,US-Canada Border,2025-06-01T00:00:00.000,Truck Containers Empty,584,47.159645,-67.930799,"{'type': 'Point', 'coordinates': [-67.930799, ..."


## Data cleaning

### Remove unneeded columns

For the purposes of our analysis, we won't need point, latitude and longitude, which contain the coordinates of the port of entry. I'm not using the port_name, state, and port_code in my analysis either, but I'm keeping them for now in case I want to drill down on a per location basis later on.

In [ ]:
# remove unneeded columns
results_df.drop(columns=["point", "latitude", "longitude"], inplace=True)
results_df

,port_name,state,port_code,border,date,measure,value
0,International Falls,Minnesota,3604,US-Canada Border,2017-01-01T00:00:00.000,Buses,18
1,Metaline Falls,Washington,3025,US-Canada Border,2017-01-01T00:00:00.000,Personal Vehicles,1857
2,Alcan,Alaska,3104,US-Canada Border,2017-01-01T00:00:00.000,Bus Passengers,0
3,Westhope,North Dakota,3419,US-Canada Border,2017-01-01T00:00:00.000,Personal Vehicles,490
4,Calais,Maine,0115,US-Canada Border,2017-01-01T00:00:00.000,Rail Containers Loaded,60
...,...,...,...,...,...,...,...
57529,Point Roberts,Washington,3017,US-Canada Border,2025-06-01T00:00:00.000,Buses,28
57530,Ogdensburg,New York,0701,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,29047
57531,Limestone,Maine,0118,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,1228
57532,Van Buren,Maine,0108,US-Canada Border,2025-06-01T00:00:00.000,Truck Containers Empty,584


### Check for NaN/null values

Next we will check for any NaN/null values

In [ ]:
# info
print("info", results_df.info())

# Null values
nulls_df = results_df[results_df.isnull().any(axis=1)]
print("Null values")
nulls_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57534 entries, 0 to 57533
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   port_name  57534 non-null  object
 1   state      57532 non-null  object
 2   port_code  57534 non-null  object
 3   border     57534 non-null  object
 4   date       57534 non-null  object
 5   measure    57534 non-null  object
 6   value      57534 non-null  object
dtypes: object(7)
memory usage: 3.1+ MB
info None
Null values


,port_name,state,port_code,border,date,measure,value
57091,Chief Mountain Mt Poe,NaN,3315,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,8844
57199,Chief Mountain Mt Poe,NaN,3315,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicles,3554


Our NaN check yielded 2 rows. It appears that the "state" field for the new port of entry "Chief Mountain Mt Poe" (port 3315) hasn't been populated yet.
We will set "state" to "Montana" for all the records with port_code == "3315"

In [ ]:

# As of 8/17/2025, the "state" field of the new port of entry "Chief Mountain Mt Poe" hasn't been populated yet.
# Will set "state" to "Montana" for all records with port_code == "3315"
results_df["port_code"] = results_df["port_code"].astype(str)
results_df.loc[results_df["port_code"] == "3315", "state"] = "Montana"

# Check null values again
print("Check null values again")
print(results_df[results_df.isnull().any(axis=1)])

print("Check rows with port code 3315")
results_df.loc[results_df["port_code"] == "3315"]

Check null values again
Empty DataFrame
Columns: [port_name, state, port_code, border, date, measure, value]
Index: []
Check rows with port code 3315


,port_name,state,port_code,border,date,measure,value
57091,Chief Mountain Mt Poe,Montana,3315,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,8844
57199,Chief Mountain Mt Poe,Montana,3315,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicles,3554


In [ ]:
# Sanity check
results_df

,port_name,state,port_code,border,date,measure,value
0,International Falls,Minnesota,3604,US-Canada Border,2017-01-01T00:00:00.000,Buses,18
1,Metaline Falls,Washington,3025,US-Canada Border,2017-01-01T00:00:00.000,Personal Vehicles,1857
2,Alcan,Alaska,3104,US-Canada Border,2017-01-01T00:00:00.000,Bus Passengers,0
3,Westhope,North Dakota,3419,US-Canada Border,2017-01-01T00:00:00.000,Personal Vehicles,490
4,Calais,Maine,0115,US-Canada Border,2017-01-01T00:00:00.000,Rail Containers Loaded,60
...,...,...,...,...,...,...,...
57529,Point Roberts,Washington,3017,US-Canada Border,2025-06-01T00:00:00.000,Buses,28
57530,Ogdensburg,New York,0701,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,29047
57531,Limestone,Maine,0118,US-Canada Border,2025-06-01T00:00:00.000,Personal Vehicle Passengers,1228
57532,Van Buren,Maine,0108,US-Canada Border,2025-06-01T00:00:00.000,Truck Containers Empty,584


## Data transformation

We want to split the date into YEAR and MONTH, which we will use to plot the graph later on. We need the MONTH in abbreviated and numeric form, for displaying and sorting, respectively.

In [ ]:
# Data transformation
results_df["date"] = pd.to_datetime(results_df["date"])
results_df["month_name"] = results_df["date"].dt.strftime('%b')
results_df["month"] = results_df["date"].dt.month
results_df["year"] = results_df["date"].dt.year
results_df["date"] = results_df["date"].dt.date


The data included commercial traffic, such as trucks, rail containers, and buses, whether or not they are carrying passengers. We are only interested in human border traffic, so we will filter for passengers and pedestrians.

In [ ]:
# Get different entry categories (measure)
entry_categories = results_df['measure'].unique()
print("entry_categories", entry_categories)

# only get passenger and pedestrian entry categories
results_df = results_df[results_df.measure.str.contains("passenger|pedestrian", na=False, case=False)]

entry_categories ['Buses' 'Personal Vehicles' 'Bus Passengers' 'Rail Containers Loaded'
 'Trucks' 'Personal Vehicle Passengers' 'Truck Containers Empty'
 'Truck Containers Loaded' 'Rail Containers Empty' 'Trains'
 'Train Passengers' 'Pedestrians']


We need to aggregate the number of crossings by year/month, so let's do that next.

In [ ]:
# Need to perform a sum on this field, will have to cast to int first
results_df.loc[:, 'value'] = results_df['value'].astype(int)
sum_by_month = results_df.groupby(['year', 'month', 'month_name'])['value'].sum().reset_index()
sum_by_month = sum_by_month.sort_values(by=['year', 'month'])
sum_by_month

,year,month,month_name,value
0,2017,1,Jan,3229664
1,2017,2,Feb,2989892
2,2017,3,Mar,3529604
3,2017,4,Apr,3871686
4,2017,5,May,4166222
...,...,...,...,...
97,2025,2,Feb,2288329
98,2025,3,Mar,2588029
99,2025,4,Apr,2400358
100,2025,5,May,2810526


Now that our data is ready, we can proceed with plotting our graph. First let's plot a graph with all the passenger and pedestrian traffic from 2017 to 2025.

In [ ]:
max_value = sum_by_month['value'].max()
figure=px.line(sum_by_month, x="month_name", y="value", color="year", symbol="year", height=800, range_y=[0 , max_value * 1.1], title="Passenger and pedestrian Canada-USA Land Border crossings from 2017 to 2025")
figure.show()

This is a lot to take in, so let's try to break it down. We can start with the years before the pandemic, from 2017 to 2019.  

In [51]:
# before covid
df_before_covid = sum_by_month[sum_by_month['year'].isin([2017, 2018, 2019])]
figure=px.line(df_before_covid,
               x="month_name",
               y="value",
               color="year",
               symbol="year",
               height=800,
               range_y=[0 , max_value * 1.1],
               title="Land crossings before the COVID pandemic, from 2017 to 2019")
figure.show()

From the graph above, we can see a pattern where there's an overall increase in crossings from the start of the year which peaks in August, and drops down with a slight uptick in December.

Traffic slightly fluctuates year to year, but follows the same pattern.

Next, we will take a look at the border crossings during the COVID pandemic. In March 2020, the US-Canadian border was closed to non-essential travel. We can see in the graph below there's a precipitous drop in March and April. Non-essential travel remained closed until August 2021, subject to proof of vaccination.

In [ ]:
# during covid (anomalous)
df_during_covid = sum_by_month[sum_by_month['year'].isin([2020, 2021])]
figure=px.line(df_during_covid, x="month_name", y="value", color="year", symbol="year", height=800, range_y=[0 , max_value * 1.1], title="Land border crossings during pandemic lockdown, 2020 to 2021")
figure.show()

Growth in traffic increased 2022 onwards, when all restrictions were lifted. However, volume is still below pre-pandemic years.

The pattern where we see a peak in the summer months was also reestablished. Growth increased on a year-on-year basis until 2025. We can see that, after January, traffic so far in 2025 has underperformed 2024, and is even below 2023.

In [ ]:
# after covid (recovery)
df_after_covid = sum_by_month[sum_by_month['year'].isin([2022, 2023, 2024, 2025])]
figure=px.line(df_after_covid, x="month_name", y="value", color="year", symbol="year", height=800, range_y=[0 , max_value * 1.1], title="Land border crossings after lockdowns were lifted, 2022 to 2025")
figure.show()

The difference is quite stark when we focus on the gap between 2024 and 2025. Traffic fell by 900k between March 2024 and 2025. By June, the gap increased to more than 1m.

In [ ]:
# closer look at 2024 and 2025
df_2024_to_2025 = sum_by_month[sum_by_month['year'].isin([2024, 2025])]
figure=px.line(df_2024_to_2025, x="month_name", y="value", color="year", symbol="year", height=800, range_y=[0 , max_value * 1.1], title="Comparison of 2024 and 2025 border crossings")
figure.show()

2025 also significantly underperforms pre-pandemic years.

Political rhetoric and reports of Canadians being detained by US officials may have contributed to the drop in travel.

In [49]:
# compare 2025 vs pre-covid years
df_current_year_vs_precovid =  sum_by_month[sum_by_month['year'].isin([2017, 2018, 2019, 2025])]
figure=px.line(df_current_year_vs_precovid, x="month_name", y="value", color="year", symbol="year", height=800, range_y=[0 , max_value * 1.1], title="Comparison of 2025 and pre-COVID border crossings")
figure.show()

In [50]:
sum_by_date = results_df.groupby(['year', 'date'])['value'].sum().reset_index()

figure=px.line(sum_by_date,
               x="date",
               y="value",
               color="year",
               symbol="year",
               height=800,
               range_y=[0 , max_value * 1.1],
               title="Land border crossings from 2017 to 2025, adjacent")
figure.show()